# 기본 예 : 프롬프트+모델+outputParser

- 저자 : [Changjun Lee] (https://www.linkedin.com/in/cjleeno1/)
-피어 리뷰 : [Erika Park] (https://www.linkedin.com/in/yeonseo-park-094193198/), [Wooseok Jeong] (https://github.com/jeong-wooseok)
- 교정 : [q011] (https://github.com/q0211)
-이것은 [Langchain Open Tutorial]의 일부입니다.

[! [열린colab] (https://colab.research.google.com/assets/colab-badge.svg)] (https://colab.research.gogle.com/github/langchain-opentutorial/langchain-opentutorial/blob/main/01-basic/06-langchain-lexpression gangugenbain-langchage (legneplygeanguge).
[! [열린github] (https://img.shields.io/badge/open%20in%20github-181717?style=flat-square&logo=github&logocolor=white)] (https : // github.com/langchain-opentutorial/langchain-opentutorial/blob/main/01-basic/06-langchain-expression-language(lcel).ipynb)


## 개요

가장 기본적이고 일반적으로 사용되는 사례는 프롬프트 템플릿을 모델과 연결하는 것입니다.이것이 어떻게 작동하는지 설명하기 위해 여러 국가의 수도를 요구하는 체인을 만들어 봅시다.


### 목차

- [개요] (#개요)
- [환경 설정] (#Environment-Setup)
-[프롬프트 템플릿 사용] (#홍보 템플릿 사용)
- [체인 생성] (#체인 제작)

### 참조

- [langchain chatopenai api 참조] (https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.html)
- [Langchain Core 출력 파서] (https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.commaseParatedListOutputparser.html#
- [Python List Tutorial] (https://docs.python.org/3.13/tutorial/datrastructures.html)
---

## 환경 설정

환경을 설정하십시오.자세한 내용은 [환경 설정] (https://wikidocs.net/257836)을 참조하십시오.

**[메모]**
-``Langchain-Opentutorial '' '는 사용하기 쉬운 환경 설정 세트, 유용한 기능 및 튜토리얼을위한 유틸리티를 제공하는 패키지입니다.
-자세한 내용은 [``````langchain-opentutorial``] (https://github.com/langchain-opentutorial/langchain-opentutorial-pypi)를 확인할 수 있습니다.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
#필요한 패키지를 설치하십시오
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
#환경 변수를 설정합니다
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
````.env '' '``````.env' ''```````````. "

[참고] 이전 단계에서 이미````openai_api_key '' '를 설정 한 경우에는 필요하지 않습니다.

In [4]:
#환경 변수로 API 키를 관리하기위한 구성 파일
from dotenv import load_dotenv

#API 키 정보를로드하십시오
load_dotenv(override=True)

True

In [5]:
#Langsmith 추적 설정 : https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

True

## 프롬프트 템플릿 사용

``PromptTemplate ''

- 프롬프트 템플릿은 사용자의 입력 변수를 통합하여 완전한 프롬프트 문자열을 만드는 데 사용됩니다.
- 용법
-```템플릿 ''`: 템플릿 문자열은 Curly Braces '{}'가 변수를 나타내는 데 사용되는 사전 정의 된 형식입니다.

-````input_variables ''«: 곱슬 괄호 안에 삽입 될 변수의 이름은 목록으로 정의됩니다.

``input_variables ''

-``input_variables '' '``````````````````````````````PromptTemplate ()는`````' '에 사용 된 변수의 이름을 정의하는 목록입니다.

In [6]:
from langchain_core.prompts import PromptTemplate

``from_template ()````````` ''객체를 작성하는 데 사용됩니다.

In [7]:
#템플릿을 정의하십시오
template = "What is the capital of {country}?"

#`from_template '메소드를 사용하여`rempttemplate'객체를 만듭니다.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [8]:
#프롬프트를 생성하십시오.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [9]:
#프롬프트를 생성하십시오.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [10]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

## 체인 생성

### lcel (Langchain 표현 언어)

여기서는 LCEN을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다.

! [lcel.png] (./ 자산/02-langchain-expression-language.png)

```
체인 = 프롬프트 |모델 |output_parser
```

```|````기호는 [unix 파이프 연산자] (<https://en.wikipedia.org/wiki/pipeline_ (unix)>)과 유사하게 작동하여 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음에 입력으로 전달합니다.

이 체인에서는 사용자 입력이 프롬프트 템플릿으로 전달되고 프롬프트 템플릿의 출력이 모델로 전달됩니다.각 구성 요소를 개별적으로 검사하면 각 단계에서 발생하는 일을 이해할 수 있습니다.

In [11]:
#'PromptTemplate'객체로 프롬프트를 만듭니다.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


#프롬프트와 모델을 체인에 결합하십시오
chain = prompt | model

### 호출```«invoke ()```

- 입력 값은 파이썬 사전 (키 값 쌍) 형태로 제공됩니다.
-````````````` ''기능을 호출 할 때 이러한 입력 값은 인수로 전달됩니다.

In [12]:
#'입력'사전에서 주제를 '인공 지능 모델에서 학습의 원리'로 설정하십시오.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [13]:
#파이프 (`|`) 연산자를 사용하여 '프롬프트'객체와 '모델'객체를 연결하십시오.
#`invoke '메소드를 사용하여'입력 '을 전달하십시오.
#AI 모델에서 생성 된 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn and make predictions. Think of it like a student learning from textbooks; the better the books, the more they learn.\n\n2. **Patterns and Features**: AI looks for patterns in the data. It identifies important features (characteristics) that help it understand the information. For example, if it’s learning to recognize cats in pictures, it might focus on features like fur texture, ear shape, and eye color.\n\n3. **Training and Testing**: AI models go through a training phase where they learn from a set of data. After training, they are tested on new data to see how well they learned. This is like practicing for a test and then taking the

아래는 스트리밍 응답을 출력하는 예입니다.

In [14]:
#스트리밍 출력 요청
answer = chain.stream(input)

#스트리밍 출력
for token in answer:
    print(token.content, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can do.

2. **Learning from Examples**: AI models learn by looking at examples. For instance, if you show an AI many pictures of cats and dogs, it can learn to tell the difference between them. This is similar to how humans learn by observing and practicing.

3. **Feedback Loop**: AI models improve through feedback. When they make mistakes, they can adjust their understanding based on the corrections. This is like a teacher giving feedback to a student to help them learn from their errors.

4. **Generalization**: AI models aim to generalize f

### 출력 파서

an ** 출력 파서 **는 AI 모델의 응답을 특정 형식으로 변환하거나 처리하도록 설계된 도구입니다.모델의 출력은 일반적으로 자유 형식 텍스트로 제공되므로 ** 출력 파서 **는 구조화 된 형식으로 변환하거나 필요한 데이터를 추출하는 데 필수적입니다.

In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
)  # Directly returns the model's response as a string without modification.

출력 파서가 체인에 추가됩니다.

In [16]:
#프롬프트, 모델 및 출력 파서를 연결하여 처리 체인이 구성됩니다.
chain = prompt | model | output_parser

In [17]:
#체인 객체의 호출 메소드를 사용하여 입력을 전달하십시오.
chain.invoke(input)

"Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can prepare.\n\n2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want to teach an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. The model learns to identify patterns that distinguish cats from other objects.\n\n3. **Feedback Loop**: AI models improve through feedback. After making predictions or decisions, they receive feedback on whether they were right or wrong. This feedback helps them adjust and learn from their mistakes, similar to h

In [18]:
#스트리밍 출력 요청
answer = chain.stream(input)

#스트리밍 출력
for token in answer:
    print(token, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can do.

2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. Over time, it learns to tell the difference.

3. **Feedback Loop**: AI models improve through feedback. When they make mistakes, they can adjust their understanding based on the corrections. This is similar to how a teacher helps a student learn from their errors.

4. **Generalization**: A good AI model can apply what it has learned to new, unseen d

### 템플릿 적용 및 수정

- 아래의 프롬프트 내용은 테스트 목적으로 필요에 따라 ** 수정 ** 될 수 있습니다.
-```model_name````````Gesting 테스트를 위해 조정할 수도 있습니다.

In [19]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#상황:
{question}

#체재:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

#PromptTemplate을 사용하여 프롬프트를 생성하십시오
prompt = PromptTemplate.from_template(template)

#Chatopenai 모델을 초기화하십시오.
model = ChatOpenAI(model_name="gpt-4o-mini")

#문자열 출력 파서를 초기화하십시오.
output_parser = StrOutputParser()

In [20]:
#체인을 구성하십시오.
chain = prompt | model | output_parser

In [21]:
#완성 된 체인을 실행하여 응답을 얻습니다.
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))

- Dialogue in English:
**Waiter:** Good evening! Welcome to La Bella Italia. How many are in your party?  
**Customer:** Just one, please.  
**Waiter:** Right this way. Here’s your menu. Can I get you something to drink while you look?  
**Customer:** Yes, I’ll have a glass of water, please.  
**Waiter:** Sure! Are you ready to order, or do you need more time?  
**Customer:** I think I’m ready. I’d like the spaghetti carbonara, please.  
**Waiter:** Excellent choice! Would you like any appetizers or desserts with that?  
**Customer:** I’ll have a side salad to start, and maybe a slice of tiramisu for dessert.  
**Waiter:** Great! I’ll put that order in for you. Anything else I can get you?  
**Customer:** No, that’s all for now, thank you.  
**Waiter:** You’re welcome! I’ll be back shortly with your order.  

- Explanation of the Dialogue: 
In this conversation, the customer arrives at a restaurant and interacts with the waiter. The waiter greets the customer and asks how many people a

In [22]:
#완성 된 체인을 실행하여 응답을 얻습니다
#스트리밍 출력 요청
answer = chain.stream({"question": "I want to go to a restaurant and order food."})

#스트리밍 출력
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:

**Customer:** Hi there! Can I get a table for two, please?  

**Host:** Sure! Right this way. Here’s your menu.  

**Customer:** Thank you! What do you recommend for a starter?  

**Host:** Our bruschetta is very popular, and the garlic shrimp is a favorite as well.  

**Customer:** Sounds delicious! We'll have the bruschetta to start.  

**Host:** Great choice! And for the main course?  

**Customer:** I’d like the grilled salmon, please. How about you?  

**Friend:** I’ll have the steak, medium rare.  

**Host:** Excellent choices! Would you like to add any sides?  

**Customer:** Yes, we’ll take a side of roasted vegetables and a Caesar salad.  

**Host:** Perfect! I’ll get that order started for you.  

**Customer:** Thank you!  

**Host:** You’re welcome! Enjoy your meal.  

---

- Explanation of the Dialogue: 

In this dialogue, the customer arrives at a restaurant and requests a table for two. The host guides them to their table and hands them the menu. T

In [23]:
#이번에는 '미국의 피자 주문'에 대한 질문을 설정하고 실행하십시오.
#스트리밍 출력 요청
answer = chain.stream({"question": "Ordering Pizza in the US"})

#스트리밍 출력
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:

**Customer:** Hi there! I’d like to order a pizza, please.

**Pizza Server:** Of course! What size would you like? We have small, medium, large, and extra-large.

**Customer:** I’ll take a large, please. 

**Pizza Server:** Great choice! What type of pizza do you want? We have pepperoni, cheese, veggie, and a few specialty pizzas.

**Customer:** I’ll go with pepperoni. Can I add extra cheese?

**Pizza Server:** Absolutely! Extra cheese on a large pepperoni pizza. Would you like anything else? 

**Customer:** Yes, can I also get a side of garlic bread and a two-liter soda?

**Pizza Server:** Sure! We have a few soda options. We have cola, diet cola, lemon-lime, and root beer. Which one would you like?

**Customer:** I’ll take a cola, please.

**Pizza Server:** Great! So that’s one large pepperoni pizza with extra cheese, a side of garlic bread, and a cola. Would you like to add any dipping sauces?

**Customer:** Yes, please! Can I get a marinara sauce and a ranch